In [1]:
import datetime
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, Reshape, Activation, Multiply, Lambda
from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

2023-06-14 14:26:36.463876: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
class LogWriterCallback(keras.callbacks.Callback):
    def __init__(self, log_path):
        super(LogWriterCallback, self).__init__()
        self.log_path = log_path

    def on_train_begin(self, logs=None):
        # This method is called at the beginning of the training.
        # It opens the log file and writes the starting time of the training.
        with open(self.log_path, 'a') as file:
            file.write('Training started at {}\n'.format(datetime.datetime.now()))

    def on_epoch_end(self, epoch, logs=None):
        # This method is called at the end of each training epoch.
        # It opens the log file and writes the epoch number and the values in the logs.
        with open(self.log_path, 'a') as file:
            file.write('Epoch {}\n'.format(epoch + 1))
            for key, value in logs.items():
                file.write('{}: {}\n'.format(key, value))
            file.write('\n')

    def on_train_end(self, logs=None):
        # This method is called at the end of the training.
        # It opens the log file and writes the finishing time of the training.
        with open(self.log_path, 'a') as file:
            file.write('Training finished at {}\n'.format(datetime.datetime.now()))


class DataGenerator(keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size, image_size):
        # Initialize the DataGenerator with the provided image paths, labels, batch size, and image size.
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size

    def __len__(self):
        # Returns the number of batches in the dataset.
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        # Generates one batch of data based on the index.
        # Get the paths and labels for the current batch.
        batch_paths = self.image_paths[index * self.batch_size: (index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size: (index + 1) * self.batch_size]

        # Preprocess the images for the current batch.
        batch_images = [self.preprocess_image(image_path) for image_path in batch_paths]
        batch_images = np.array(batch_images)

        # Return the preprocessed batch of images and corresponding labels.
        return batch_images, batch_labels

    def preprocess_image(self, image_path):
        # Preprocesses a single image given its path.
        # Load the image and resize it to the specified image size.
        image = keras.preprocessing.image.load_img(image_path, target_size=self.image_size)

        # Convert the image to a numpy array.
        image = keras.preprocessing.image.img_to_array(image)

        # Normalize the pixel values to the range [0, 1].
        image = image / 255.0

        # Return the preprocessed image.
        return image


def load_dataset(data_set_path, data_csv_path):
    # Load the data from the CSV file using pandas read_csv function.
    data_df = pd.read_csv(data_csv_path)

    # Remove rows where the "Finding Labels" column has the value 'No Finding',
    # as they represent cases with no findings and are not relevant for this dataset.
    data_df = data_df[data_df["Finding Labels"] != 'No Finding'].reset_index(drop=True)

    # Create a list of image paths by joining the data set path with the "Image Index" column in the DataFrame.
    image_paths = [os.path.join(data_set_path, row['Image Index']) for _, row in data_df[['Image Index']].iterrows()]

    # Split the "Finding Labels" column by '|' to obtain a list of labels for each image.
    labels = data_df["Finding Labels"].apply(lambda x: x.split("|"))

    # Return the image paths and labels.
    return image_paths, labels


def preprocess_labels(labels, mlb=None):
    # Preprocesses the labels using MultiLabelBinarizer.
    # If mlb is not provided, a new MultiLabelBinarizer instance is created and fitted on the labels.
    if mlb is None:
        mlb = MultiLabelBinarizer()
        labels = mlb.fit_transform(labels)
    else:
        # If mlb is provided, transform the labels using the existing MultiLabelBinarizer instance.
        labels = mlb.transform(labels)

    # Get the number of unique labels.
    num_labels = len(mlb.classes_)

    # Reshape the labels to have shape (-1, num_labels), where -1 means the size is inferred based on the data.
    labels = np.reshape(labels, (-1, num_labels))

    # Return the preprocessed labels, the MultiLabelBinarizer instance, and the number of labels.
    return labels, mlb, num_labels


def build_model_Res(input_shape, num_labels):
    # Build a model using the ResNet50 architecture.

    # Load the pre-trained ResNet50 model with weights from 'imagenet'.
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Get the output tensor of the base model.
    x = base_model.output

    # Add a GlobalAveragePooling2D layer to reduce spatial dimensions.
    x = GlobalAveragePooling2D()(x)

    # Add a fully connected Dense layer with 1024 units and ReLU activation.
    x = Dense(1024, activation='relu')(x)

    # Add a final Dense layer with num_labels units and sigmoid activation for multi-label classification.
    predictions = Dense(num_labels, activation='sigmoid')(x)

    # Create the model by specifying the inputs and outputs.
    model = Model(inputs=base_model.input, outputs=predictions)

    # Return the built model.
    return model


def build_model_with_attention_Res(input_shape, num_labels, attention_units=256):
    # Builds a model based on ResNet50 architecture with attention mechanism.

    # Load the pre-trained ResNet50 model with weights trained on ImageNet.
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Get the output tensor from the base model.
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Attention mechanism
    attention = Dense(attention_units, activation='tanh')(x)
    attention = Dense(1, activation='sigmoid')(attention)
    attention = Reshape((-1,))(attention)
    attention = Multiply()([x, attention])
    attention = Reshape((1, 1, -1))(attention)
    attention = GlobalAveragePooling2D()(attention)

    # Fully connected layers for classification
    x = Dense(1024, activation='relu')(attention)
    predictions = Dense(num_labels, activation='sigmoid')(x)

    # Create the model with base_model.input as the input and predictions as the output.
    model = Model(inputs=base_model.input, outputs=predictions)

    # Return the model
    return model


def build_model_with_self_attention_Res(input_shape, num_labels, attention_units=256):
    # Builds a model based on ResNet50 architecture with self attention mechanism.

    # Load the pre-trained ResNet50 model with weights trained on ImageNet.
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    # Get the output tensor from the base model.
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Self-Attention mechanism
    attention = Reshape((1, 1, -1))(x)
    attention = Conv2D(filters=attention_units, kernel_size=1, padding='same', activation='relu')(attention)
    attention = Conv2D(filters=1, kernel_size=1, padding='same', activation='sigmoid')(attention)
    attention = Lambda(lambda x: tf.reduce_mean(x, axis=[1, 2]))(attention)
    attention = Activation('softmax')(attention)
    attention = Reshape((-1, 1))(attention)
    attention = Multiply()([x, attention])
    attention = Lambda(lambda x: tf.reduce_mean(x, axis=1))(attention)
    # Fully connected layers for classification
    x = Dense(1024, activation='relu')(attention)
    predictions = Dense(num_labels, activation='sigmoid')(x)
    # Create the model with base_model.input as the input and predictions as the output.
    model = Model(inputs=base_model.input, outputs=predictions)
    return model



def build_model_with_self_attention_Dense(input_shape, num_labels, attention_units=256):
    # Builds a model based on DenseNet150 architecture with self attention mechanism.

    # Load the pre-trained DenseNet150 model with weights trained on ImageNet.
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    # Get the output tensor from the base model.
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Self-Attention mechanism
    attention = Reshape((1, 1, -1))(x)
    attention = Conv2D(filters=attention_units, kernel_size=1, padding='same', activation='relu')(attention)
    attention = Conv2D(filters=1, kernel_size=1, padding='same', activation='sigmoid')(attention)
    attention = Lambda(lambda x: tf.reduce_mean(x, axis=[1, 2]))(attention)
    attention = Activation('softmax')(attention)
    attention = Reshape((-1, 1))(attention)
    attention = Multiply()([x, attention])
    attention = Lambda(lambda x: tf.reduce_mean(x, axis=1))(attention)
    # Fully connected layers for classification
    x = Dense(1024, activation='relu')(attention)
    predictions = Dense(num_labels, activation='sigmoid')(x)
    # Create the model with base_model.input as the input and predictions as the output.
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

def build_model_with_categorical_self_attention_Dense(input_shape, num_labels, attention_units=256):
    # Builds a model based on DenseNet150 architecture with categorical wise self attention mechanism.

    # Load the pre-trained DenseNet150 model with weights trained on ImageNet.
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    # Get the output tensor from the base model.
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Categorical-wise Self-Attention mechanism
    num_categories = int(x.shape[-1])
    attention = Reshape((1, 1, num_categories))(x)
    attention = Conv2D(filters=attention_units, kernel_size=1, padding='same', activation='relu')(attention)
    attention = Conv2D(filters=num_categories, kernel_size=1, padding='same', activation='sigmoid')(attention)
    attention = Lambda(lambda x: tf.reduce_mean(x, axis=[1, 2]))(attention)
    attention = Activation('softmax')(attention)
    attention = Reshape((-1, 1))(attention)
    attention = Multiply()([x, attention])
    attention = Lambda(lambda x: tf.reduce_mean(x, axis=1))(attention)
    # Fully connected layers for classification
    x = Dense(1024, activation='relu')(attention)
    predictions = Dense(num_labels, activation='sigmoid')(x)
    # Create the model with base_model.input as the input and predictions as the output.
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


def train_model(model, train_generator, val_generator, epochs, callbacks):

    # This line compiles the model with the Adam optimizer, binary crossentropy loss, and accuracy and AUC metrics.
    """
    Compiles the model with the specified optimizer, loss, and metrics.

    Args:
        model: The model to compile.
        optimizer: The optimizer to use.
        loss: The loss function to minimize.
        metrics: The metrics to monitor during training and evaluation.

    Returns:
        The compiled model.
    """

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

    # This line fits the model to the training data, using the validation data for validation. The epochs parameter specifies the number of times to train the model on the training data. The callbacks parameter specifies a list of callbacks that will be called during training.
    """
    Fits the model to the training data, using the validation data for validation.

    Args:
        model: The model to fit.
        train_generator: The training data generator.
        val_generator: The validation data generator.
        epochs: The number of epochs to train the model.
        callbacks: A list of callbacks to be called during training.

    Returns:
        The history of the model training.
    """

    model.fit(train_generator, validation_data=val_generator, epochs=epochs, callbacks=callbacks)

def evaluate_model(model, test_generator):

    # This line evaluates the model on the test data and prints the loss, accuracy, and AUC metrics.

    """
    Evaluates the model on the test data and prints the loss, accuracy, and AUC metrics.

    Args:
        model: The model to evaluate.
        test_generator: The test data generator.

    Returns:
        The loss, accuracy, and AUC metrics.
    """

    loss, accuracy, auc = model.evaluate(test_generator)
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)
    print("Test AUC:", auc)


# Set the paths and parameters
data_set_path = "./datasets/NIH/images/"
dataset_folder = os.path.join("./", "nih_dataset")
data_csv_path = os.path.join(dataset_folder, "Data_Entry_2017.csv")
batch_size = 16
image_size = (224, 224)

# Load the dataset metadata
image_paths, labels = load_dataset(data_set_path, data_csv_path)

# Perform train-validation-test split
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)
train_paths, val_paths, train_labels, val_labels = train_test_split(train_paths, train_labels, test_size=0.1, random_state=42)

# Convert labels to binary vectors
train_labels, mlb, num_labels = preprocess_labels(train_labels)
val_labels, _, _ = preprocess_labels(val_labels, mlb)
test_labels, _, _ = preprocess_labels(test_labels, mlb)

# Create data generators
train_generator = DataGenerator(train_paths, train_labels, batch_size, image_size)
val_generator = DataGenerator(val_paths, val_labels, batch_size, image_size)
test_generator = DataGenerator(test_paths, test_labels, batch_size, image_size)

# Build the model

#model = build_model_Res(input_shape=(224, 224, 3), num_labels=num_labels)
#model = build_model_with_attention_Res(input_shape=(224, 224, 3), num_labels=num_labels)
#model = build_model_with_self_attention_Res(input_shape=(224, 224, 3), num_labels=num_labels)

model = build_model_with_self_attention_Dense(input_shape=(224, 224, 3), num_labels=num_labels)
#model =  build_model_with_categorical_self_attention_Dense(input_shape=(224, 224, 3), num_labels=num_labels)

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_auc', patience=10, mode='max', verbose=1)

# Define the log path
approach_name = "self_attention_dense"

log_path = 'training_{}_log.txt'.format(approach_name)
# Define the log writer callback
log_writer = LogWriterCallback(log_path)

# Define the checkpoint callback
checkpoint_path = 'best_model_{}.h5'.format(approach_name)
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_auc', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint,early_stopping,log_writer]

# Train the model
train_model(model, train_generator, val_generator, epochs= 500, callbacks=callbacks)


2023-06-14 14:26:40.815917: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-14 14:26:40.826900: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-14 14:26:40.827138: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/500


2023-06-14 14:26:44.425092: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-14 14:26:57.287888: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 4090" frequency: 2520 num_cores: 128 environment { key: "architecture" value: "8.9" } environment { key: "cuda" value: "11080" } environment { key: "cudnn" value: "8600" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 75497472 shared_memory_size_per_multiprocessor: 102400 memory_size: 23151050752 bandwidth: 1008096000 } outputs { dtype: DT_FLOAT shape { u

2330/2330 [==============================] - ETA: 0s - loss: 0.2922 - accuracy: 0.2896 - auc: 0.7895

2023-06-14 14:33:08.382817: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_auc improved from -inf to 0.80475, saving model to best_model_self_attention_dense.h5
2330/2330 [==============================] - 425s 165ms/step - loss: 0.2922 - accuracy: 0.2896 - auc: 0.7895 - val_loss: 0.2882 - val_accuracy: 0.3308 - val_auc: 0.8048
Epoch 2/500
2330/2330 [==============================] - ETA: 0s - loss: 0.2772 - accuracy: 0.3302 - auc: 0.8199
Epoch 2: val_auc did not improve from 0.80475
2330/2330 [==============================] - 377s 162ms/step - loss: 0.2772 - accuracy: 0.3302 - auc: 0.8199 - val_loss: 0.2952 - val_accuracy: 0.2869 - val_auc: 0.7977
Epoch 3/500
2330/2330 [==============================] - ETA: 0s - loss: 0.2696 - accuracy: 0.3458 - auc: 0.8335
Epoch 3: val_auc did not improve from 0.80475
2330/2330 [==============================] - 372s 160ms/step - loss: 0.2696 - accuracy: 0.3458 - auc: 0.8335 - val_loss: 0.2900 - val_accuracy: 0.3086 - val_auc: 0.8039
Epoch 4/500
2330/2330 [==============================] - ETA: 0s - loss: 0.

In [ ]:
# Load the best model
best_model = keras.models.load_model(checkpoint_path)

# Evaluate the best model on the test set
evaluate_model(best_model, test_generator)

# Calculate the average AUC on the test set
test_pred_prob = best_model.predict(test_generator)
test_true_labels = mlb.transform(test_labels)
auc_scores = []
for i in range(num_labels):
    auc = roc_auc_score(test_true_labels[:, i], test_pred_prob[:, i])
    auc_scores.append(auc)
average_auc = np.mean(auc_scores)
print("Average AUC:", average_auc)